In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ks_2samp
import scipy.stats as stats
from scipy.stats import ttest_rel

In [2]:
data = pd.read_excel(r'CATALIST UK EXXON V4 2019.xlsx')

In [6]:
(data.isnull().sum().sort_values(ascending=False))/len(data)

Electric Charging Brands   0.98
Suburb 2                   0.97
Separate Catering          0.95
In Store Deli Brand        0.94
Secondary Road Number      0.94
LPG Brand                  0.91
Car Sales                  0.86
In Store Catering          0.85
Secondary Street           0.82
Building Number            0.71
Car Wash Brand             0.54
Jet Wash Units             0.54
Automatic Wash Units       0.54
Conveyor Wash Units        0.54
ATM Name                   0.46
Suburb                     0.39
Coffee Offer Brand         0.30
Primary Road Number        0.22
Shop Sales                 0.11
Shop Area                  0.11
Shop Brand                 0.11
Hand Car Wash              0.05
Customer Toilets           0.04
TPS OPT-OUT                0.04
Telephone                  0.04
CTPS OPT-OUT               0.04
Company Name               0.03
MF Volume                  0.02
Closing Time               0.01
Opening Time               0.01
                           ... 
Microwav

In [7]:
df_tesco = pd.read_pickle(r'df_tesco_marker_V3.pkl')

df_tesco.columns

In [9]:
cols = ['Site Number', 'Site Name','Site ID', 'PBL']
df_site = df_tesco.loc[:,cols]
df_site.drop_duplicates(keep='first', inplace=True)

In [26]:
df_tesco.columns

Index(['Site Number', 'Site Name', 'Transaction Date', 'Total Sales Value',
       'Total Baskets', 'Sales Baskets With Fuel', 'Baskets With Fuel',
       'Site ID', 'PBL', 'CRUSO_priceADO', 'CRUSO_priceUL95', 'volumeADO',
       'volumeUL95', 'sales_valADO', 'sales_valUL95', 'totalSalesEsso',
       'pct_dev', 'diff_shop_esso', 'SiteCode', 'SiteName', 'Date',
       'UL 95_ Sales Transaction Volume', 'UL 95_ Approved Price',
       'UL 95_ Marker Price', 'UL 95_ Approved Price Vs Marker Price',
       'ADO_ Sales Transaction Volume', 'ADO_ Approved Price',
       'ADO_ Marker Price', 'ADO_ Approved Price Vs Marker Price',
       'avgUL95VolSite', 'avgADOVolSite', 'diffAdoVol', 'diffUL95Vol',
       'diffAdoVolpct', 'diffUL95Volpct', 'diffAdoPricePct',
       'diffUL95PricePct', 'avgShopSalesFromFuelSiteWise',
       'pctChgShopSalesFromFuel', 'shopSales', 'avgShopSalesSiteWise',
       'ShopSalesPctChgFromAvg', 'AdoApprPriceChgPct', 'UL95AppPriceChgPct',
       'avgFuelSalesEssoSiteWi

In [18]:
data['Cat Number']

0           1
1           3
2           5
3           9
4          11
5          13
6          15
7          17
8          18
9          19
10         21
11         23
12         25
13         26
14         29
15         31
16         41
17         43
18         46
19         48
20         51
21         54
22         56
23         57
24         64
25         66
26         68
27         72
28         73
29         76
        ...  
7975    26351
7976    26352
7977    26353
7978    26354
7979    26355
7980    26356
7981    26357
7982    26358
7983    26362
7984    26363
7985    26364
7986    26365
7987    26366
7988    26367
7989    26368
7990    26371
7991    26372
7992    26374
7993    26376
7994    26377
7995    26378
7996    26379
7997    26380
7998    26381
7999    26382
8000    26383
8001    26386
8002    26387
8003    26388
8004    26389
Name: Cat Number, dtype: int64

In [20]:
df_m = pd.merge(df_site,data,how='left',left_on = 'PBL',right_on='Cat Number')

In [21]:
df_m.isnull().sum()

Site Number                0
Site Name_x                0
Site ID                    0
PBL                        0
Cat Number               196
Country Code             196
Brand                    196
Site Name_y              196
Building Number          196
Street                   196
Secondary Street         196
Suburb                   196
Suburb 2                 196
Town                     196
COUNTY                   196
Region                   196
Local Authority Area     196
Postcode                 196
Country                  196
Primary Road Number      196
Secondary Road Number    196
Company Name             196
Telephone                196
Positioned X             196
Positioned Y             196
Site Status              196
Site Type                196
Petrol                   196
Diesel                   196
LPG                      196
                        ... 
Shop Quality             196
Dedicated Spaces         196
Pedestrian Customers     196
Shop Competiti

In [25]:
df_tesco['weekday'] = df_tesco['Transaction Date'].dt.strftime('%A')

In [54]:
weekday_sales = df_tesco.groupby(['Site Name','Site Number','PBL','weekday'])['exclShopSales'].agg({'median':'median'}).unstack().reset_index()

weekday_sales.columns = ['Site Name','Site Number','PBL','Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']

In [45]:
weekday_sales

Site Name  Site Number        PBL    Friday    Monday  \
0    ABBEY ST ACCRNGTN EXP         5350 200,911.00 10,007.22  8,240.45   
1         ABINGTON EXPRESS         2005 200,740.00  6,399.33  5,287.61   
2    ACKLAM RD MIDSBRO EXP         5254 328,841.00 15,648.87 12,812.79   
3            ACTON EXPRESS         2012 300,931.00 12,245.61 11,162.06   
4       ADDISCOMBE EXPRESS         2004 201,307.00 13,240.78 11,670.79   
5          ARDSLEY EXPRESS         5462 200,810.00  8,399.08  6,639.67   
6        ARDWICK GREEN EXP         5435 200,108.00  7,465.47  7,383.21   
7    ASHTON IN MAKRFLD EXP         5081 200,386.00  8,594.70  6,690.19   
8           AULD CROSS EXP         5669 200,227.00  9,613.28  8,747.78   
9           BALHAM EXPRESS         2211 309,618.00 13,701.19 11,752.65   
10         BARKING EXPRESS         2058 300,187.00  9,180.11  8,307.00   
11          BARNES EXPRESS         2066 301,694.00 10,658.42  9,835.31   
12         BASFORD EXPRESS         2183 200,253.00  6,177.30  5,257.99   
13          BATTLE EXPRESS         2177 200,656.00 12,441.70 10,445.84   
14        BEARSTED EXPRESS         5346 200,874.00 11,780.52  9,243.86   
15   BESSACARR DONCAST EXP         5665 200,854.00  9,818.92  7,849.33   
16   BEWDLEY HILL KIDD EXP         5165 200,103.00  9,380.06  7,282.48   
17      BILLY MILL EXPRESS         2968 200,765.00  4,504.32  3,376.52   
18        BITTERNE EXPRESS         2175 200,848.00  8,643.89  7,133.25   
19   BOLTON BRADFRD ST EXP         5604 200,005.00  9,403.52  7,409.45   
20   BOOTHFERRY HESSLE EXP         5497 200,143.00  9,175.39  7,075.59   
21          BOURNE EXPRESS         2192 201,150.00  6,572.58  5,291.40   
22     BOURNVILLE BHAM EXP         5386 334,145.00  6,697.53  6,700.94   
23   BRAMHALL CHESHIRE EXP         5204 200,914.00  9,787.80  7,679.69   
24         BRECHIN EXPRESS         5413 200,506.00  6,305.57  4,911.82   
25       BRENTWOOD EXPRESS         2176 201,207.00 14,291.10 11,938.97   
26     BRISTL SOUNDWEL EXP         5582 201,046.00  8,823.95  7,517.65   
27      BRISTOL FILTON EXP         2492 200,229.00  9,125.93  8,070.76   
28         BROMLEY EXPRESS         2203 200,566.00 13,789.62 11,841.21   
29   BULLSMRE LN WAL X EXP         3361 300,185.00  9,961.22  8,954.08   
..                     ...          ...        ...       ...       ...   
166    STORRINGTON EXPRESS         3199 200,954.00 11,492.23  9,538.37   
167  TARBOCK WOODLANDS EXP         5791 200,321.00 11,845.44  9,329.69   
168       TOLWORTH EXPRESS         3295 200,985.00 14,691.49 12,510.65   
169       TONBRIDGE RD EXP         2902 200,878.00 12,312.59 10,106.42   
170        TORQUAY EXPRESS         3296 200,746.00  6,617.90  5,389.03   
171      TULSEHILL EXPRESS         3317 200,902.00  8,362.13  7,203.53   
172        TUPSLEY EXPRESS         3318 200,330.00  8,707.26  7,291.95   
173  TWYFORD RD EASTLG EXP         5250 339,412.00  9,490.54  7,996.74   
174    WALTHAMSTOW EXPRESS         3359 321,050.00  8,201.03  7,832.21   
175            WARMLEY EXP         5679 201,045.00 13,575.88 11,413.43   
176    WEDNESFIELD EXPRESS         3374 200,694.00  8,738.43  7,328.30   
177   WEST BYFLEET EXPRESS         3381 200,572.00  7,362.91  6,256.80   
178   WEST WICKHAM EXPRESS         3386 201,390.00  7,358.55  6,436.76   
179           WESTBURN EXP         5959 200,089.00  8,259.52  7,993.58   
180  WEYMOUTH LANEH RD EXP         5434 201,198.00  7,212.87  5,861.86   
181     WHITCHURCH EXPRESS         3424 200,463.00  9,781.40  7,537.47   
182     WHITEGATES EXPRESS         5424 200,582.00 12,708.88 10,606.30   
183  WHITEHOUS SCOLDFD EXP         5098 200,995.00  7,635.47  6,456.29   
184    WIGAN RD BOLTON EXP         5251 328,844.00  8,781.41  7,769.51   
185    WILLENHALL BHAM EXP         5278 328,840.00  8,180.09  6,474.89   
186      WIMBLEDON EXPRESS         3402 309,617.00 11,661.29 10,206.31   
187  WINCHMORE HILL EXPRES         3393 300,371.00  5,700.46  4,852.57   
188  WINTERBOURN BRST

In [38]:
PDI_elasticity_results = pd.read_excel(r'pdi_elaticity_values.xlsx')

In [56]:
weekday_sales = weekday_sales.merge(pdi,how = 'left',left_on='PBL',right_on='DisplayId')

In [47]:
PDI_elasticity_results.columns

Index(['DisplayId', 'CatalistId', 'Grade', 'GradeId', 'Elasticity'], dtype='object')

In [57]:
weekday_sales.isnull().sum()

Site Name      0
Site Number    0
PBL            0
Friday         0
Monday         0
Saturday       2
Sunday         2
Thursday       0
Tuesday        0
Wednesday      0
DisplayId      0
ADO            0
UL 95          0
dtype: int64

In [58]:
weekday_sales

Site Name  Site Number        PBL    Friday    Monday  \
0    ABBEY ST ACCRNGTN EXP         5350 200,911.00 10,007.22  8,240.45   
1         ABINGTON EXPRESS         2005 200,740.00  6,399.33  5,287.61   
2    ACKLAM RD MIDSBRO EXP         5254 328,841.00 15,648.87 12,812.79   
3            ACTON EXPRESS         2012 300,931.00 12,245.61 11,162.06   
4       ADDISCOMBE EXPRESS         2004 201,307.00 13,240.78 11,670.79   
5          ARDSLEY EXPRESS         5462 200,810.00  8,399.08  6,639.67   
6        ARDWICK GREEN EXP         5435 200,108.00  7,465.47  7,383.21   
7    ASHTON IN MAKRFLD EXP         5081 200,386.00  8,594.70  6,690.19   
8           AULD CROSS EXP         5669 200,227.00  9,613.28  8,747.78   
9           BALHAM EXPRESS         2211 309,618.00 13,701.19 11,752.65   
10         BARKING EXPRESS         2058 300,187.00  9,180.11  8,307.00   
11          BARNES EXPRESS         2066 301,694.00 10,658.42  9,835.31   
12         BASFORD EXPRESS         2183 200,253.00  6,177.30  5,257.99   
13          BATTLE EXPRESS         2177 200,656.00 12,441.70 10,445.84   
14        BEARSTED EXPRESS         5346 200,874.00 11,780.52  9,243.86   
15   BESSACARR DONCAST EXP         5665 200,854.00  9,818.92  7,849.33   
16   BEWDLEY HILL KIDD EXP         5165 200,103.00  9,380.06  7,282.48   
17      BILLY MILL EXPRESS         2968 200,765.00  4,504.32  3,376.52   
18        BITTERNE EXPRESS         2175 200,848.00  8,643.89  7,133.25   
19   BOLTON BRADFRD ST EXP         5604 200,005.00  9,403.52  7,409.45   
20   BOOTHFERRY HESSLE EXP         5497 200,143.00  9,175.39  7,075.59   
21          BOURNE EXPRESS         2192 201,150.00  6,572.58  5,291.40   
22     BOURNVILLE BHAM EXP         5386 334,145.00  6,697.53  6,700.94   
23   BRAMHALL CHESHIRE EXP         5204 200,914.00  9,787.80  7,679.69   
24         BRECHIN EXPRESS         5413 200,506.00  6,305.57  4,911.82   
25       BRENTWOOD EXPRESS         2176 201,207.00 14,291.10 11,938.97   
26     BRISTL SOUNDWEL EXP         5582 201,046.00  8,823.95  7,517.65   
27      BRISTOL FILTON EXP         2492 200,229.00  9,125.93  8,070.76   
28         BROMLEY EXPRESS         2203 200,566.00 13,789.62 11,841.21   
29   BULLSMRE LN WAL X EXP         3361 300,185.00  9,961.22  8,954.08   
..                     ...          ...        ...       ...       ...   
166    STORRINGTON EXPRESS         3199 200,954.00 11,492.23  9,538.37   
167  TARBOCK WOODLANDS EXP         5791 200,321.00 11,845.44  9,329.69   
168       TOLWORTH EXPRESS         3295 200,985.00 14,691.49 12,510.65   
169       TONBRIDGE RD EXP         2902 200,878.00 12,312.59 10,106.42   
170        TORQUAY EXPRESS         3296 200,746.00  6,617.90  5,389.03   
171      TULSEHILL EXPRESS         3317 200,902.00  8,362.13  7,203.53   
172        TUPSLEY EXPRESS         3318 200,330.00  8,707.26  7,291.95   
173  TWYFORD RD EASTLG EXP         5250 339,412.00  9,490.54  7,996.74   
174    WALTHAMSTOW EXPRESS         3359 321,050.00  8,201.03  7,832.21   
175            WARMLEY EXP         5679 201,045.00 13,575.88 11,413.43   
176    WEDNESFIELD EXPRESS         3374 200,694.00  8,738.43  7,328.30   
177   WEST BYFLEET EXPRESS         3381 200,572.00  7,362.91  6,256.80   
178   WEST WICKHAM EXPRESS         3386 201,390.00  7,358.55  6,436.76   
179           WESTBURN EXP         5959 200,089.00  8,259.52  7,993.58   
180  WEYMOUTH LANEH RD EXP         5434 201,198.00  7,212.87  5,861.86   
181     WHITCHURCH EXPRESS         3424 200,463.00  9,781.40  7,537.47   
182     WHITEGATES EXPRESS         5424 200,582.00 12,708.88 10,606.30   
183  WHITEHOUS SCOLDFD EXP         5098 200,995.00  7,635.47  6,456.29   
184    WIGAN RD BOLTON EXP         5251 328,844.00  8,781.41  7,769.51   
185    WILLENHALL BHAM EXP         5278 328,840.00  8,180.09  6,474.89   
186      WIMBLEDON EXPRESS         3402 309,617.00 11,661.29 10,206.31   
187  WINCHMORE HILL EXPRES         3393 300,371.00  5,700.46  4,852.57   
188  WINTERBOURN BRST

In [53]:
pdi = PDI_elasticity_results.pivot_table(index=['DisplayId'],columns='Grade',values='Elasticity').reset_index()

In [59]:
weekday_sales.to_excel('excl_site_sales_weekday.xlsx')